# Декораторы и их объяснение

## Базовые концепции для начала понимания

In [ ]:
# 1. Функции можно присваивать переменным
def greet(name):
    return f"Hello, {name}!"

my_function = greet
print(my_function("Alice"))  # "Hello, Alice!"

In [ ]:
# 2. Функции можно передавать как аргументы
def call_twice(func, arg):
    """Вызывает функцию дважды с одним аргументом"""
    return func(arg) + " " + func(arg)

result = call_twice(greet, "Bob")
print(result)

Замыкания

In [ ]:
# 3. Функции можно возвращать из функций
def create_greeter(greeting):
    def greeter(name):
        return f"{greeting}, {name}!"
    return greeter

hello = create_greeter("Hello")
print(hello("Charlie"))

## Концепция декоратора

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Что-то происходит перед вызовом функции")
        func()
        print("Что-то происходит после вызова функции")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

# Эквивалентно: say_hello = my_decorator(say_hello)

say_hello()

### Передача аргументов в декоратор

In [ ]:
from functools import wraps


def repeat(num_times):
    """Декоратор, который повторяет вызов функции заданное количество раз"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            results = []
            for _ in range(num_times):
                result = func(*args, **kwargs)
                results.append(result)
            return results
        return wrapper
    return decorator

@repeat(num_times=3)
def greet(name):
    return f"Hello, {name}!"

print(greet("Alice"))

## Примеры декораторов из практики

### Измерение времени работы

In [ ]:
import time
from functools import wraps

def timer(func):
    """Декоратор, который измеряет время выполнения функции"""
    @wraps(func)  # Сохраняем метаданные оригинальной функции
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Функция {func.__name__} выполнилась за {end_time - start_time:.4f} секунд")
        return result
    return wrapper

@timer
def slow_function():
    """Функция, которая имитирует долгую операцию"""
    time.sleep(2)
    return "Готово!"

print(slow_function())

### Логгирование функции

In [ ]:
from functools import wraps

def logger(func):
    """Декоратор для логирования вызовов функции"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Вызов функции: {func.__name__}")
        print(f"Аргументы: args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"Результат: {result}")
        return result
    return wrapper

@logger
def calculate(a, b, operation='add'):
    if operation == 'add':
        return a + b
    elif operation == 'multiply':
        return a * b

calculate(5, 3, operation='multiply')

### Кеширование функции

Есть отдельный декоратор в Python, однако бывают случаи, когда нужно реализовать кастомную логику

In [ ]:
from functools import wraps

def cache(func):
    """Декоратор для кэширования результатов функции"""
    cache_dict = {}
    
    @wraps(func)
    def wrapper(*args):
        if args in cache_dict:
            print(f"Возвращаем кэшированный результат для {args}")
            return cache_dict[args]
        else:
            print(f"Вычисляем результат для {args}")
            result = func(*args)
            cache_dict[args] = result
            return result
    return wrapper

@cache
def fibonacci(n):
    """Вычисляет n-ное число Фибоначчи"""
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))

### Нестандартный способ создания декоратора

In [ ]:
from functools import wraps


class CountCalls:
    """Декоратор как класс, который считает вызовы функции"""
    def __init__(self, func):
        self.func = func
        self.num_calls = 0
        wraps(func)(self)  # Сохраняем метаданные

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Вызов #{self.num_calls} функции {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print("Hello!")

say_hello()  # Вызов #1 функции say_hello
say_hello()  # Вызов #2 функции say_hello
print(f"Всего вызовов: {say_hello.num_calls}")  # 2